In [1]:
import json
import os
import re
import requests
from openplugincore import OpenPlugin
import openai
import logging
from typing import Any, Callable, Dict, List, Optional, TypedDict
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

# filter out "Attempting to load an OpenAPI 3.0.0 spec..." from log
with open('migration.log', 'r') as file:
    lines = file.readlines()
filtered_lines = [line for line in lines if 'your undesired string' not in line]
with open('migration.log', 'w') as file:
    file.writelines(filtered_lines)

class OpenPluginInfoBase(TypedDict):
    namespace: str
    image: Optional[str]
    description_for_human: Optional[str]
    description_for_model: str
    domain: str
    openapi_url: str

class OpenPluginInfoInit(OpenPluginInfoBase, TypedDict):
    whitelisted: bool

class OpenPluginInfoStimulated(OpenPluginInfoInit, TypedDict):
    stimulated: bool
    stimulous_prompt: Optional[str]

class OpenPluginInfo(OpenPluginInfoStimulated, TypedDict):
    status: str

OpenPluginsInfo = Dict[str, OpenPluginInfo]

logging.basicConfig(filename='migration.log', level=logging.INFO, format='%(asctime)s %(message)s')

openplugin_classes: Dict[str, OpenPlugin] = {}

# 1. Assign `all_plugins.json` to variable `all_plugins`
pypi_client_path = 'openai_res.json'
with open(pypi_client_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
try:
    json_content = ''.join(lines)
    openai_res = json.loads(json_content)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
print(f"Number of plugins: {len(openai_res)}")

# 2. Create `blacklist_list` dict
if os.path.exists('blacklist.json'):
    with open('blacklist.json', 'r', encoding='utf-8') as f:
        blacklist = json.load(f)
else:
    blacklist = {}
    with open('blacklist.json', 'w', encoding='utf-8') as f:
        json.dump(blacklist, f, indent=2)
def aggregate_items(json_object):
    aggregate_list = []
    for key in json_object:
        aggregate_list.extend(json_object[key])
    return aggregate_list
blacklist_list = aggregate_items(blacklist)
print(f"Number of blacklisted plugins: {len(blacklist_list)}")

# 3. Assign `plugins_info.json` to variable `plugins_info`
if os.path.exists('openplugins_info.json'):
    with open('openplugins_info.json', 'r', encoding='utf-8') as f:
        openplugins_info: dict = json.load(f)
else:
    openplugins_info = {}
    with open('openplugins_info.json', 'w', encoding='utf-8') as f:
        json.dump(openplugins_info, f, indent=2)
        
print(f"Number of plugins in openplugins_info: {len(openplugins_info)}")

Number of plugins: 700
Number of blacklisted plugins: 34
Number of plugins in openplugins_info: 681


In [2]:
openai_res_ex = {
  "id": "plugin-027a8b9d-7f54-42d3-8a04-1b6391997cf8",
  "domain": "plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app",
  "namespace": "Ai_PDF",
  "status": "approved",
  "manifest": {
    "schema_version": "v1",
    "name_for_model": "Ai_PDF",
    "name_for_human": "Ai PDF",
    "description_for_model": "Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.",
    "description_for_human": "Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.",
    "auth": {
      "type": "none"
    },
    "api": {
      "type": "openapi",
      "url": "https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml"
    },
    "logo_url": "https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png",
    "contact_email": "support@promptapps.ai",
    "legal_info_url": "https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/legal.html"
  },
  "oauth_client_id": None,
  "user_settings": {
    "is_installed": True,
    "is_authenticated": True
  },
  "categories": [
    {
      "id": "most_popular",
      "title": "Most popular"
    }
  ]
}

# openai_res_ex = {
#   "id": "plugin-988f53b4-5d81-4703-983a-38b6dac4b8ac",
#   "domain": "chat-web3-plugin.alchemy.com",
#   "namespace": "Alchemy",
#   "status": "approved",
#   "manifest": {
#     "schema_version": "v1",
#     "name_for_model": "Alchemy",
#     "name_for_human": "Alchemy",
#     "description_for_model": "Request real-time blockchain data for chains like Ethereum, Polygon, Arbitrum and Optimism through natural language.",
#     "description_for_human": "Request real-time blockchain data for chains like Ethereum, Polygon, Arbitrum and Optimism through natural language.",
#     "auth": {
#       "type": "service_http",
#       "instructions": "",
#       "authorization_type": "bearer",
#       "verification_tokens": {
#         "openai": "f9cd21497220486789fa94c7a129a040"
#       }
#     },
#     "api": {
#       "type": "openapi",
#       "url": "https://chat-web3-plugin.alchemy.com/openapi.yaml"
#     },
#     "logo_url": "https://chat-web3-plugin.alchemy.com/logo.png",
#     "contact_email": "support@alchemy.com",
#     "legal_info_url": "https://www.alchemy.com/policies/terms"
#   },
#   "oauth_client_id": None,
#   "user_settings": {
#     "is_installed": False,
#     "is_authenticated": True
#   },
#   "categories": [
#     {
#       "id": "newly_added",
#       "title": "New"
#     }
#   ]
# }

In [3]:

def generate_base_opinfo(openplugins_info: OpenPluginsInfo, blacklist_list: List[str],  openai_res_plugin: Any) -> OpenPluginInfoBase:
    namespace = openai_res_plugin['namespace']
    openplugin_info = {
        **openplugins_info.get(namespace, {}),
        'namespace': namespace,
        'image': openai_res_plugin['manifest'].get('logo_url', None),
        'description_for_human': openai_res_plugin['manifest'].get('description_for_human', None),
        'description_for_model': openai_res_plugin['manifest']['description_for_model'],
        'domain': openai_res_plugin['domain'],
        'openapi_url': openai_res_plugin['manifest']['api']['url']
    }
    # auth check
    if openai_res_plugin['manifest'].get('auth', {}).get('type', None) == 'none':
        openplugin_info['auth'] = False
    else:
        openplugin_info['auth'] = True
    # blacklisted check
    if openai_res_plugin['namespace'] in blacklist_list:
        openplugin_info['blacklisted'] = True
    else:
        openplugin_info['blacklisted'] = False
    openplugins_info[namespace] = openplugin_info

    return openplugin_info

first = generate_base_opinfo(openplugins_info, blacklist_list, openai_res_ex)
first

{'namespace': 'Ai_PDF',
 'image': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png',
 'description_for_human': 'Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.',
 'description_for_model': 'Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.',
 'domain': 'plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app',
 'openapi_url': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml',
 'auth': False,
 'blacklisted': False,
 'whitelisted': True,
 'stimulous_prompt': 'Search a PDF https://americanenglish.state.gov/files/ae/resource_files/the_black_cat.pdf',
 'stimulated': True,
 'status': 'supported',
 'js_info': {'whitelisted': True, 'stimulated': True, 'status': 'supported'}}

In [4]:
def test_fetch_and_parse(openplugins_info: OpenPluginsInfo, openplugin_classes: Dict[str, OpenPlugin], openai_res_plugin: Any) -> OpenPluginInfoInit: 
    namespace = openai_res_plugin['namespace']
    openplugin_info = openplugins_info[namespace]
    root_url = "https://" + openplugin_info['domain']
    try:
        openplugin_classes[namespace] = OpenPlugin(openai_api_key=OPENAI_API_KEY, root_url=root_url)
        logging.info(f"{namespace} Whitelist Success")
        openplugin_info['whitelisted'] = True
    except Exception as e:
        logging.error(f"{namespace} Whitelist Error: {e}")
        openplugin_info['whitelisted'] = False

    return openplugin_info

second = test_fetch_and_parse(openplugins_info, openplugin_classes, openai_res_ex)
second

{'namespace': 'Ai_PDF',
 'image': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png',
 'description_for_human': 'Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.',
 'description_for_model': 'Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.',
 'domain': 'plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app',
 'openapi_url': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml',
 'auth': False,
 'blacklisted': False,
 'whitelisted': True,
 'stimulous_prompt': 'Search a PDF https://americanenglish.state.gov/files/ae/resource_files/the_black_cat.pdf',
 'stimulated': True,
 'status': 'supported',
 'js_info': {'whitelisted': True, 'stimulated': True, 'status': 'supported'}}

In [5]:
def test_stimulate(openplugins_info: OpenPluginsInfo, openplugin_classes: Dict[str, OpenPlugin], openai_res_plugin: Any) -> OpenPluginInfoStimulated:
    namespace = openai_res_plugin['namespace']
    openplugin_info = openplugins_info[namespace]
    openplugin_class = openplugin_classes[namespace]
    try:
        generate_stimulation_prompt_prompt = {
            "prompt": f"""
            Please create a prompt that will trigger an model's plugin with the human description delimited by driple backticks.
            If necessary also look at the model description also delimited by triple backticks.
            Please do not ask anything from the AI you should provide all the information it needs in the prompt.
            You should not be ambiguous or open ended in your prompt use specific examples.
            Do not simply restate the description.
            Human description:
            ```
            {openplugin_info["description_for_human"]}
            ```
            Model description:
            ```
            {openplugin_info["description_for_model"]}
            ```
            """,
            "function": {
            "name": "stimulous_prompt_generation",
            "description": """
            Generates a natural language phrase to that triggeres the AI plugin.
            If approriate the phrase should include an example item/url (https://github.com/)/text/ect. even if you are not sure if it is real its ok to make it up.
            """,
            "parameters": {
                "type": "object",
                "properties": {
                "stimulous_prompt": {
                    "type": "string",
                    "description": "The stimulous phrase to trigger the AI plugin"
                },
                },
                "required": ["stimulous_prompt"]
            }
            }
        }
        generation = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            temperature=0.7,
            messages=[{"role": "user", "content": generate_stimulation_prompt_prompt["prompt"]}],
            functions=[generate_stimulation_prompt_prompt["function"]],
            function_call={"name": "stimulous_prompt_generation"}
        )
        json_arguments = json.loads(generation["choices"][0]["message"]["function_call"]["arguments"])
        openplugin_info["stimulous_prompt"] = json_arguments["stimulous_prompt"]
        openplugin_class.fetch_plugin(
            prompt=openplugin_info["stimulous_prompt"],
            model="gpt-3.5-turbo-0613",
            temperature=0,
        )
        openplugin_info["stimulated"] = True
        logging.info(f"{namespace} Stimulate Success")
    except Exception as e:
        openplugin_info["stimulous_prompt"] = openplugin_info.get("stimulous_prompt", None)
        openplugin_info["stimulated"] = False
        logging.error(f"{namespace} Stimulate Error: {e}")
    return openplugin_info

third = test_stimulate(openplugins_info, openplugin_classes, openai_res_ex)
third

{'namespace': 'Ai_PDF',
 'image': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png',
 'description_for_human': 'Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.',
 'description_for_model': 'Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.',
 'domain': 'plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app',
 'openapi_url': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml',
 'auth': False,
 'blacklisted': False,
 'whitelisted': True,
 'stimulous_prompt': "Search for information in a PDF document and provide page references. For example, search for 'fact checking' in a PDF about technology trends. The document is located at 'https://example.com/technology-trends.pdf'.",
 'stimulated': True,
 'status': 'supported',
 'js_info': {'whitelisted': True, 'stimulated

In [6]:
def assign_status(openplugins_info: OpenPluginsInfo, openai_res_plugin: Any) -> OpenPluginInfo:
    namespace = openai_res_plugin['namespace']
    openplugin_info = openplugins_info[namespace]
    openplugin_info["status"] = "unsupported"
    if not openplugin_info["auth"] and not openplugin_info["blacklisted"] and openplugin_info["whitelisted"]:
        openplugin_info["status"] = "tentative"
    if openplugin_info["stimulous_prompt"] and openplugin_info["stimulated"]:
        openplugin_info["status"] = "supported"

    return openplugin_info

fourth = assign_status(openplugins_info, openai_res_ex)
fourth    

{'namespace': 'Ai_PDF',
 'image': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png',
 'description_for_human': 'Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.',
 'description_for_model': 'Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.',
 'domain': 'plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app',
 'openapi_url': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml',
 'auth': False,
 'blacklisted': False,
 'whitelisted': True,
 'stimulous_prompt': "Search for information in a PDF document and provide page references. For example, search for 'fact checking' in a PDF about technology trends. The document is located at 'https://example.com/technology-trends.pdf'.",
 'stimulated': True,
 'status': 'supported',
 'js_info': {'whitelisted': True, 'stimulated

In [7]:
def save_openplugin(openplugins_info: OpenPluginsInfo, openai_res_plugin: Any) -> None:
    # save to openplugins_info.json
    if not os.path.exists('openplugins_info.json'):
        with open('openplugins_info.json', 'w', encoding='utf-8') as f:
            json.dump(openplugins_info, f, indent=2)
    else:
        with open('openplugins_info.json', 'w', encoding='utf-8') as f:
            json.dump(openplugins_info, f, indent=2)
    
    # save to openplugins.json
    namespace = openai_res_plugin['namespace']
    openplugin_info = openplugins_info[namespace]
    if (openplugin_info["status"] == "supported" or openplugin_info["status"] == "tentative"):
        with open('openplugins.json', 'r', encoding='utf-8') as f:
            openplugins = json.load(f)
        openplugins[namespace] = "https://" + openplugin_info["domain"]
        with open('openplugins.json', 'w', encoding='utf-8') as f:
            json.dump(openplugins, f, indent=2)
        logging.info(f"{namespace} Save to openplugins Success")
    else:
        logging.info(f"{namespace} Save to openplugins Skipped")
fifth = save_openplugin(openplugins_info, openai_res_ex)
fifth

In [9]:
def save_to_pluginsmd(openplugins_info: OpenPluginsInfo) -> None:
    current_dir = os.getcwd()
    root_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

    # Create the file path for plugins.md
    pluginsmd_path = os.path.join(root_dir, 'PLUGINS.md')
    plugins_md = """# Plugins
Available plugins for OpenPlugin
Status:
- `tentative`: passed basic tests (may work)
- `supported`: passed complete prompt tests (should work)

| Image | Namespace | Status | Description | Description for model |
| --- | --- | --- | --- | --- |
"""

    def escape_special_markdown_chars(text):
        # Characters to escape: \ ` * _ { } [ ] ( ) # + !
        special_chars = r'\\|`|\*|_|{|}|\[|\]|\(|\)|#|\+|!'
        return re.sub(special_chars, lambda match: '\\' + match.group(), text)

    def remove_line_breaks(text):
        return text.replace('\n', ' ').replace('\r', '')

    # create two lists of plugins one supported_plugins and one tentative_plugins
    supported_plugins = []
    tentative_plugins = []
    for _namespace, openplugin_info in openplugins_info.items():
        if openplugin_info["status"] == "tentative":
            tentative_plugins.append(openplugin_info)
        if openplugin_info["status"] == "supported":
            supported_plugins.append(openplugin_info)
    # now sort the lists by their namespace keys considering that each plugin is a dict of {namespace: str, ...}
    supported_plugins.sort(key=lambda x: x["namespace"])
    tentative_plugins.sort(key=lambda x: x["namespace"])
    # aggragate the lists so that supported_plugins is first and tentative_plugins is second 
    ordered_openplugins_info = supported_plugins + tentative_plugins

    for openplugin_info in ordered_openplugins_info:
            if openplugin_info["image"]:
                image = escape_special_markdown_chars(openplugin_info["image"])
            else:
                image = escape_special_markdown_chars("https://i.imgur.com/L3giCRt.png")
            namespace = escape_special_markdown_chars(openplugin_info["namespace"])
            status = escape_special_markdown_chars(openplugin_info["status"])
            description = escape_special_markdown_chars(remove_line_breaks(remove_line_breaks(openplugin_info["description_for_human"])))
            description_for_model = escape_special_markdown_chars(remove_line_breaks(openplugin_info["description_for_model"]))
            plugins_md += f"| ![{namespace} Logo]({image}) | {namespace} | {status} | {description} | {description_for_model} |\n"

    with open(pluginsmd_path, 'w', encoding='utf-8') as f:
        f.write(plugins_md)

sixth = save_to_pluginsmd(openplugins_info)
sixth


In [10]:
def classify_plugin(openplugins_info: OpenPluginsInfo, openai_res_plugin: Any) -> OpenPluginInfo:
    # first
    openplugin_info = generate_base_opinfo(openplugins_info, blacklist_list, openai_res_plugin)
    # second
    if openplugin_info["auth"] or openplugin_info["blacklisted"] or openplugin_info.get("whitelisted", None) == False:
        openplugin_info["whitelisted"] = False
        openplugin_info["stimulous_prompt"] = None
        openplugin_info["stimulated"] = False
    else:
        openplugin_info = test_fetch_and_parse(openplugins_info, openplugin_classes, openai_res_plugin)
        if openplugin_info["whitelisted"]:
            openplugin_info = test_stimulate(openplugins_info, openplugin_classes, openai_res_plugin)
        else:
            openplugin_info["stimulous_prompt"] = None
            openplugin_info["stimulated"] = False
    openplugin_info = assign_status(openplugins_info, openai_res_plugin)
    return openplugin_info
composite = classify_plugin(openplugins_info, openai_res_ex)
composite

{'namespace': 'Ai_PDF',
 'image': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png',
 'description_for_human': 'Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.',
 'description_for_model': 'Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.',
 'domain': 'plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app',
 'openapi_url': 'https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml',
 'auth': False,
 'blacklisted': False,
 'whitelisted': True,
 'stimulous_prompt': 'Generate a chat with a PDF of any size and provide page references for fact checking. Use the following URL: https://example.com/sample.pdf',
 'stimulated': True,
 'status': 'supported',
 'js_info': {'whitelisted': True, 'stimulated': True, 'status': 'supported'}}

In [11]:
def cleanup_log() -> None:
    with open('migration.log', 'r') as file:
        lines = file.readlines()
    filtered_lines = [line for line in lines if 'Attempting to load an OpenAPI 3.' not in line]
    with open('migration.log', 'w') as file:
        file.writelines(filtered_lines)

In [13]:
# remove all duplicates from openai_res.json
def cleanup_duplicates_openai_res(openai_res: List[Any]):
    openai_res_no_duplicates = []
    for openai_res_plugin in openai_res:
        if openai_res_plugin not in openai_res_no_duplicates:
            openai_res_no_duplicates.append(openai_res_plugin)
    with open('openai_res.json', 'w', encoding='utf-8') as f:
        json.dump(openai_res_no_duplicates, f, indent=2)
    print(f"Original amt: {len(openai_res)}\nNew amt: {len(openai_res_no_duplicates)}")
    return len(openai_res_no_duplicates)
cleanup_duplicates_openai_res(openai_res)


Original amt: 700
New amt: 695


695

In [20]:
a={"a":{}}
bool(a["a"] and a["a"]["b"])

False

In [22]:
# iterate trhough all items in the openai_res.json array
start_idx = 0
only_new = True
for idx, openai_res_plugin in enumerate(openai_res[start_idx:]):
    if only_new and openplugins_info.get(openai_res_plugin['namespace'], False):
        continue
    prorated_idx = idx + start_idx
    if prorated_idx % 20 == 0:
        print(f"{prorated_idx} of {len(openai_res)}")
    logging.info(f"{prorated_idx} Processing {openai_res_plugin['namespace']}")
    openplugin_info = classify_plugin(openplugins_info, openai_res_plugin)
    save_openplugin(openplugins_info, openai_res_plugin)
    logging.info(f"{openai_res_plugin['namespace']} Complete")
    cleanup_log()
save_to_pluginsmd(openplugins_info)

In [23]:
num_unsupported = 0
num_tentative = 0
num_supported = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["status"] == "unsupported":
        num_unsupported += 1
    if openplugin_info["status"] == "tentative":
        num_tentative += 1
    if openplugin_info["status"] == "supported":
        num_supported += 1
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"{len(openplugins_info)} total plugins\n{num_unsupported} plugins with status unsupported\n{num_tentative} plugins with status tentative\n{num_supported} plugins with status supported\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")

684 total plugins
307 plugins with status unsupported
198 plugins with status tentative
179 plugins with status supported
First openplugin info: {
  "namespace": "Ai_PDF",
  "image": "https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png",
  "description_for_human": "Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.",
  "description_for_model": "Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.",
  "domain": "plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app",
  "openapi_url": "https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml",
  "auth": false,
  "blacklisted": false,
  "whitelisted": true,
  "stimulous_prompt": "Generate a chat with a PDF of any size and provide page references for fact checking. Use the following URL: https://example.com/sample.pdf",
  "stimul

In [25]:
# first run js-classify
with open('openplugins_info.json', 'r', encoding='utf-8') as f:
    openplugins_info: dict = json.load(f)
def assign_js_status_to_all(openplugins_info):
    for _namespace, openplugin_info in openplugins_info.items():
        if openplugin_info["js_info"]["whitelisted"] and openplugin_info["js_info"]["stimulated"]:
            openplugin_info["js_info"]["status"] = "supported"
        if openplugin_info["js_info"]["whitelisted"] and not openplugin_info["js_info"]["stimulated"]:
            openplugin_info["js_info"]["status"] = "tentative"
        if not openplugin_info["js_info"]["whitelisted"]:
            openplugin_info["js_info"]["status"] = "unsupported"

assign_js_status_to_all(openplugins_info)
with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2)

js_num_supported = 0
js_num_tentative = 0
js_num_unsupported = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["js_info"]["status"] == "unsupported":
        js_num_unsupported += 1
    if openplugin_info["js_info"]["status"] == "tentative":
        js_num_tentative += 1
    if openplugin_info["js_info"]["status"] == "supported":
        js_num_supported += 1
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"{len(openplugins_info)} total plugins\n{js_num_unsupported} plugins with status unsupported\n{js_num_tentative} plugins with status tentative\n{js_num_supported} plugins with status supported\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")

684 total plugins
306 plugins with status unsupported
256 plugins with status tentative
122 plugins with status supported
First openplugin info: {
  "namespace": "Ai_PDF",
  "image": "https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/logo.png",
  "description_for_human": "Super-fast, interactive chats with PDFs of any size, complete with page references for fact checking.",
  "description_for_model": "Provide a URL to a PDF and search the document. Break the user question in multiple semantic search queries and calls as needed. Think step by step.",
  "domain": "plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app",
  "openapi_url": "https://plugin-3c56b9d4c8a6465998395f28b6a445b2-jexkai4vea-uc.a.run.app/openapi.yaml",
  "auth": false,
  "blacklisted": false,
  "whitelisted": true,
  "stimulous_prompt": "Generate a chat with a PDF of any size and provide page references for fact checking. Use the following URL: https://example.com/sample.pdf",
  "stimul